# Proceso ETL para obtener la serie completa de natalidad (1975-2023)

In [1]:
import pandas as pd
import re
from scipy.interpolate import interp1d
import numpy as np

In [ ]:
#Natalidad 
natalidad=pd.read_csv("/home/pablo/Documentos/CSVs TFG/Natalidad_bruta_nacional.csv", sep=";")
natalidad=natalidad.drop(columns=["Totales Territoriales"])
natalidad=natalidad.rename(columns={"Total": "Tasa Natalidad bruta"})
natalidad

,Periodo,Tasa Natalidad bruta
0,2023,"6,61"
1,2022,"6,88"
2,2021,"7,1"
3,2020,"7,19"
4,2019,"7,62"
5,2018,"7,94"
6,2017,"8,41"
7,2016,"8,8"
8,2015,"9,02"
9,2014,"9,18"


In [ ]:
natalidad.iloc[:,0]=pd.to_numeric(natalidad.iloc[:,0], errors='coerce')
natalidad.iloc[:,1]=natalidad.iloc[:,1].str.replace(',', '.').astype(float)
natalidad.iloc[:,1]=pd.to_numeric(natalidad.iloc[:,1], errors='coerce')

In [6]:
natalidad

,Periodo,Tasa Natalidad bruta
0,2023,6.61
1,2022,6.88
2,2021,7.1
3,2020,7.19
4,2019,7.62
5,2018,7.94
6,2017,8.41
7,2016,8.8
8,2015,9.02
9,2014,9.18


In [ ]:
def linear_interpolation_between_two_points(x1, y1, x2, y2, num_points):
    
    #Genero puntos equidistantes entre x1 y x2
    x_values=np.linspace(x1, x2, num_points)
    
    #Calculo los valores interpolados
    y_values=y1 + (y2 - y1) * (x_values - x1) / (x2 - x1)
    
    return x_values, y_values

In [ ]:
def rellenar_natalidad(i=0,num_points=13):
    total_x=[]
    total_y=[]

    while i<len(natalidad)-1:

        try:
            x1, y1=natalidad.iloc[i,0], natalidad.iloc[i,1]
            x2, y2=natalidad.iloc[i+1,0], natalidad.iloc[i+1,1]
        except:
            print("error")

        interpolated_x_values, interpolated_y_values=linear_interpolation_between_two_points(x1, y1, x2, y2, num_points)
        
        if i!=len(natalidad)-2:
            interpolated_x_values=np.delete(interpolated_x_values, -1)
            interpolated_y_values=np.delete(interpolated_y_values, -1)

        total_x.append(interpolated_x_values)
        total_y.append(interpolated_y_values)


        i=i+1
        
        #print(diciembre)
        #print(lista)
    resultado_x=np.concatenate(total_x, axis=0)
    resultado_y=np.concatenate(total_y, axis=0)
    return resultado_x, resultado_y

In [9]:
ano_x,natalidad_y=rellenar_natalidad()

In [10]:
ano_x=list(ano_x)

In [11]:
x=0
while x<len(ano_x):
    ano_x[x]=str(ano_x[x])
    x=x+1

In [12]:
diccionario_valores={".0":".12",
                      ".9166666666667":".11",
                        ".8333333333333":".10",
                        ".75":".9",
                        ".6666666666667":".8",
                        ".5833333333333":".7",
                        ".5":".6",
                        ".4166666666667":".5",
                        ".3333333333333":".4",
                        ".25":".3",
                        ".1666666666667":".2",
                        ".0833333333333":".1"}

In [13]:
x=0
nuevo_ano=[]

while x<len(ano_x):

    if ano_x[x][4:]!=".0":
        xaumento=int(ano_x[x].split(".")[0])+1
        xreal=diccionario_valores[f"{ano_x[x][4:]}"]
        xfinal=f"{xaumento}{xreal}"
    else:
        xsinaumento=ano_x[x].split(".")[0]
        xreal=diccionario_valores[f"{ano_x[x][4:]}"]
        xfinal=f"{xsinaumento}{xreal}"

    nuevo_ano.append(xfinal)

    x=x+1
print(nuevo_ano)

['2023.12', '2023.11', '2023.10', '2023.9', '2023.8', '2023.7', '2023.6', '2023.5', '2023.4', '2023.3', '2023.2', '2023.1', '2022.12', '2022.11', '2022.10', '2022.9', '2022.8', '2022.7', '2022.6', '2022.5', '2022.4', '2022.3', '2022.2', '2022.1', '2021.12', '2021.11', '2021.10', '2021.9', '2021.8', '2021.7', '2021.6', '2021.5', '2021.4', '2021.3', '2021.2', '2021.1', '2020.12', '2020.11', '2020.10', '2020.9', '2020.8', '2020.7', '2020.6', '2020.5', '2020.4', '2020.3', '2020.2', '2020.1', '2019.12', '2019.11', '2019.10', '2019.9', '2019.8', '2019.7', '2019.6', '2019.5', '2019.4', '2019.3', '2019.2', '2019.1', '2018.12', '2018.11', '2018.10', '2018.9', '2018.8', '2018.7', '2018.6', '2018.5', '2018.4', '2018.3', '2018.2', '2018.1', '2017.12', '2017.11', '2017.10', '2017.9', '2017.8', '2017.7', '2017.6', '2017.5', '2017.4', '2017.3', '2017.2', '2017.1', '2016.12', '2016.11', '2016.10', '2016.9', '2016.8', '2016.7', '2016.6', '2016.5', '2016.4', '2016.3', '2016.2', '2016.1', '2015.12', '201

In [14]:
data={"Periodo":nuevo_ano,"Tasa Natalidad bruta":natalidad_y}
natalidad=pd.DataFrame(data)

In [15]:
natalidad

,Periodo,Tasa Natalidad bruta
0,2023.12,6.6100
1,2023.11,6.6325
2,2023.10,6.6550
3,2023.9,6.6775
4,2023.8,6.7000
...,...,...
572,1976.4,18.7100
573,1976.3,18.7075
574,1976.2,18.7050
575,1976.1,18.7025


In [ ]:
#Guardo en CSV
natalidad.to_csv("SerieCompletaNatalidad.csv", index=False)

In [ ]:
def convertir_periodo_a_fecha(df, columna_periodo="Periodo"):
    df[columna_periodo]=df[columna_periodo].astype(str).apply(lambda x: f"{x.split('.')[0]}-{int(x.split('.')[1]):02d}")

    df[columna_periodo]=pd.to_datetime(df[columna_periodo], format='%Y-%m') + pd.offsets.MonthEnd(1)

    return df

In [18]:
natalidad=convertir_periodo_a_fecha(natalidad)

In [20]:
natalidad.to_csv("SerieCompletaNatalidadConFechas.csv", index=False)

In [21]:
len(natalidad["Periodo"])

577